In [ ]:
import pdal
import json


pc_file = "demo/AOI-PointCloud.laz"


pdal_pipeline = [
    pc_file,
    # {"type": "filters.info"},
]
pipeline = pdal.Pipeline(json.dumps(pdal_pipeline))
pipeline.execute()

# metadata = pipeline.metadata["metadata"]
metadata = pipeline.metadata
print(json.dumps(metadata, indent=4))


In [24]:
import rasterio
from rasterio.warp import transform_bounds
from shapely.geometry import box
from shapely.affinity import scale
import matplotlib.pyplot as plt

dsm_file = "demo/AOI-DigitalSurfaceModel.tif"
with rasterio.open(dsm_file) as dsm:
    bbox = dsm.bounds
    poly = box(*bbox)
    buffered_poly = scale(poly, xfact=2, yfact=2)
    transformed = transform_bounds(dsm.crs, {"init": "EPSG:4326"}, *buffered_poly.bounds)

    print(bbox)
    print(*bbox)
    print(poly)
    print(buffered_poly)
    print(list(buffered_poly.bounds))
    print(transformed)

# fig = plt.figure(1)
# ax = fig.add_subplot()
# ax.plot(*buffered_poly.exterior.xy)
# ax.plot(*poly.exterior.xy)
# plt.show()


BoundingBox(left=627316.5, bottom=4323079.0, right=627779.5, top=4323511.0)
627316.5 4323079.0 627779.5 4323511.0
POLYGON ((627779.5 4323079, 627779.5 4323511, 627316.5 4323511, 627316.5 4323079, 627779.5 4323079))
POLYGON ((628011 4322863, 628011 4323727, 627085 4323727, 627085 4322863, 628011 4322863))
[627085.0, 4322863.0, 628011.0, 4323727.0]
(-85.53145646709716, 39.04546515284791, -85.52059690334895, 39.05338353990129)


In [5]:
from odc.stac import stac_load
import planetary_computer as pc
from pystac_client import Client
from IPython.display import HTML, display

x, y = (-76.490798, 40.280593)
r = 0.05
bbox = (x - r, y - r, x + r, y + r)

catalog = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

query = catalog.search(
    collections=["3dep-lidar-dsm"],
    bbox=bbox
)

items = list(query.get_items())
print(f"Found: {len(items):d} datasets")


Found: 4 datasets


In [26]:
urls = [i.assets["data"].href for i in items]
urls = [pc.sign(h) for h in urls]


In [2]:
from typing import List, Optional, Tuple
import requests
from pathlib import Path
from tempfile import TemporaryDirectory
from collections import Counter
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.merge import merge


def download_dsms(urls: List[str], destination: str) -> List[str]:
    local_paths = []
    for url in urls:
        download_path = Path(destination, Path(url).name)
        response = requests.get(url, stream=True)
        with open(download_path, "wb") as fout:
            for chunk in response.iter_content(chunk_size=4096):
                fout.write(chunk)
        local_paths.append(download_path)
    return local_paths


def make_consistent(dsm_paths: List[str], destination: str) -> List[str]:
    """Does not account differing resolutions"""
    crss = []
    for dsm_path in dsm_paths:
        with rasterio.open(dsm_path) as src:
            crss.append(src.crs)

    crs_count = Counter(crss)
    if crs_count.__len__() > 1:
        dst_crs = crs_count.most_common(1)[0][0]

    consistent_paths = []
    for dsm_path in dsm_paths:
        with rasterio.open(dsm_path) as src:
            if src.crs != dst_crs:
                consistent_path = dsm_path.parent / f"{dsm_path.stem}_reprojected.tif"
                # https://rasterio.readthedocs.io/en/latest/topics/reproject.html
                transform, width, height = calculate_default_transform(
                    src.crs, dst_crs, src.width, src.height, *src.bounds
                )
                kwargs = src.meta.copy()
                kwargs.update({
                    'crs': dst_crs,
                    'transform': transform,
                    'width': width,
                    'height': height
                })
                with rasterio.open(consistent_path, 'w', **kwargs) as dst:
                    reproject(
                        source=rasterio.band(src, 1),
                        destination=rasterio.band(dst, 1),
                        src_transform=src.transform,
                        src_crs=src.crs,
                        dst_transform=transform,
                        dst_crs=dst_crs,
                        resampling=Resampling.nearest
                    )
                consistent_paths.append(consistent_path)
            else:
                consistent_paths.append(dsm_path)
    return consistent_dsm_paths


def merge_crop_dsms(paths: List[str], destination: str) -> str:
    open_dsms = [rasterio.open(dsm_path) for dsm_path in dsm_paths]
    merged, transform = merge(open_dsms)

    out_meta = open_dsms[0].meta.copy()
    for open_dsm in open_dsms:
        open_dsm.close()
    out_meta.update({
        "driver": "GTiff",
        "height": merged.shape[1],
        "width": merged.shape[2],
        "transform": transform
    })

    merged_path = Path(destination, "merged.tif")
    with rasterio.open(merged_path, "w", **out_meta) as dst:
        dst.write(merged)

    return merged_path


urls = [pc.sign(i.assets["data"].href) for i in items]

with TemporaryDirectory() as tmp_dir:
    dsm_paths = download_dsms(urls, tmp_dir)
    if len(dsm_paths) > 1:
        consistent_dsm_paths = make_consistent(dsm_paths, tmp_dir)
        merged_dsm = merge_crop_dsms(consistent_dsm_paths, tmp_dir)
    # crop
    # upload to s3


IndentationError: expected an indented block (1242351508.py, line 39)

In [25]:
from rasterio.merge import merge
import rasterio
from collections import Counter
from tempfile import TemporaryDirectory
from pathlib import Path
import requests

# 1. download all images
dsms = {}
for item in items:
    dsms[item.id] = {}
    dsms[item.id]["url"] = pc.sign(item.assets["data"].href)


with TemporaryDirectory() as tmp_dir:
    for key, value in dsms.items():
        download_path = Path(tmp_dir, Path(value["url"]).name)
        response = requests.get(value["url"], stream=True)
        with open(download_path, "wb") as fout:
            for chunk in response.iter_content(chunk_size=4096):
                fout.write(chunk)
        dsms[key]["temp_path"] = download_path

    # 2. enforce constant CRS
    if len(dsms) > 1:
        for key, dsm in dsms.items():
            with rasterio.open(dsm["temp_path"]) as src:
                dsms[key]["crs"] = src.crs
        
        crs_count = Counter([dsm["crs"] for dsm in dsms.values()])
        if crs_count.__len__() > 1:
            dst_crs = crs_count.most_common(1)[0][0]
            for key, dsm in dsms.items():
                with rasterio.open(dsm["temp_path"]) as src:
                    if src.crs != dst_crs:
                        # reproject: https://rasterio.readthedocs.io/en/latest/topics/reproject.html




# 2. if more than one image:

    # 2a. enforce constant CRS

    # 2b. merge into single image: https://rasterio.readthedocs.io/en/latest/api/rasterio.merge.html
    #                              https://automating-gis-processes.github.io/CSC18/lessons/L6/raster-mosaic.html

# 3. crop to bounding box: https://rasterio.readthedocs.io/en/latest/topics/windowed-rw.html

# 4. upload to S3



# open_dsms = [rasterio.open(dsm_href) for dsm_href in dsm_hrefs]
# merged, transform = merge(open_dsms, bbox)
# print(merged.shape)
# print(transform)

COMPD_CS["NAD83 / UTM zone 18N + NAVD88 height",PROJCS["NAD83 / UTM zone 18N",GEOGCS["NAD83",DATUM["North_American_Datum_1983",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6269"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4269"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-75],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","26918"]],VERT_CS["NAVD88 height",VERT_DATUM["North American Vertical Datum 1988",2005,AUTHORITY["EPSG","5103"]],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Gravity-related height",UP],AUTHORITY["EPSG","5703"]]]


In [23]:
x = [3, 2, 3]
max(x)

3